Naive Bayes classifier that can separate spam and non-spam emails. By Damilola Yinusa

In [ ]:
pip install scikit-learn pandas

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import re
import os

# Function to clean text by removing special characters and converting to lowercase
def clean_text(text):
    return re.sub(r'[^\w\s]', '', text).lower()

# Check if the dataset file exists
if not os.path.exists('email_data.csv'):
    print("The file 'email_data.csv' does not exist.")
else:
    # Read dataset from CSV file
    df = pd.read_csv('spam_ham_dataset.csv')

    # Clean the email text
    df['text'] = df['text'].apply(clean_text)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

    # Create a TF-IDF vectorizer with bigrams and unigrams, and fit_transform the training data
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Define a Random Forest Classifier
    params = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 30, 50, 100],
        'class_weight': ['balanced']
    }
    clf = RandomForestClassifier(random_state=42)
    grid_clf = GridSearchCV(clf, params, cv=5)
    
    # Train the classifier
    grid_clf.fit(X_train_tfidf, y_train)
    
    # Make predictions
    y_pred = grid_clf.predict(X_test_tfidf)

    # Evaluate the classifier
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Precision: {precision_score(y_test, y_pred)}')
    print(f'Recall: {recall_score(y_test, y_pred)}')
    print(f'F1 Score: {f1_score(y_test, y_pred)}')
    print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
